In [2]:
#packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import themepy
from highlight_text import HighlightText, ax_text, fig_text
from adjustText import adjust_text
import matplotlib.font_manager
from IPython.core.display import HTML
import glob
import os

In [3]:
#fonts
def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

In [4]:
#setting existing theme
theme = themepy.Theme()
theme.set_theme("pavel_dark")

pavel_dark is the active theme

In [21]:
#Loading files
all_files = glob.glob(os.path.join("...\\Teams", "*.csv"))
teams= pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
teams.reset_index(drop=True,inplace=True)

all_files2 = glob.glob(os.path.join("...\\Outfield", "*.csv"))
df= pd.concat((pd.read_csv(f) for f in all_files2), ignore_index=True)
df.reset_index(drop=True,inplace=True)

In [24]:
#df operations
df=df[df['position'].str.contains("MF")]
df.reset_index(drop=True,inplace=True)

full=pd.merge(df,teams,on="squad",how="left")
full=full[full["minutes_x"]>1000]
full.reset_index(drop=True,inplace=True)

In [26]:
#calculating per90 stats and ratings
df=pd.DataFrame()
df['player']=full.player
df['position']=full.position
df['squad']=full.squad
df['age_full']=full.age
df["age"]=df.age_full.apply(lambda x:str(x.split("-")[0]))
df['minutes_x']=full.minutes_x
df['minutes_y']=full.minutes_y
df["npxg"]=full.npxg_per90_x/full.npxg_per90_y*full.npxg_per90_y.mean()
df["xa"]=full.xa_per90_x/full.xa_per90_y*full.xa_per90_y.mean()
df["touches"]=(full.touches_x/full.minutes_x*90)/(full.touches_y/full.minutes_y*90)*(full.touches_y.mean()/full.minutes_y*90)
df["cross_tend"]=full.crosses_x/full.touches_mid_3rd_x
df["drib_tend"]=full.dribbles_x/full.touches_x
df["pr_passes"]=(full.progressive_passes_x/full.minutes_x*90)/(full.progressive_passes_y/full.minutes_y*90)*(full.progressive_passes_y.mean()/full.minutes_y*90)
df["pr_carries"]=(full.progressive_carries_x/full.minutes_x*90)/(full.progressive_carries_y/full.minutes_y*90)*(full.progressive_carries_y.mean()/full.minutes_y*90)
temp=df["npxg"].rank(pct=True)*0.125+df["xa"].rank(pct=True)*0.125+df["touches"].rank(pct=True)*0.2+df["cross_tend"].rank(pct=True)*0.1+df["drib_tend"].rank(pct=True)*0.05+df["pr_passes"].rank(pct=True)*0.3+df["pr_carries"].rank(pct=True)*0.1
df["rating_off"]=temp
df["pressures"]=(full.pressures_att_3rd_x/full.minutes_x*90)/(full.pressures_att_3rd_y/full.minutes_y*90)*(full.pressures_att_3rd_y.mean()/full.minutes_y*90)
df["tackles"]=(full.tackles_att_3rd_x/full.minutes_x*90)/(full.tackles_att_3rd_y/full.minutes_y*90)*(full.tackles_att_3rd_y.mean()/full.minutes_y*90)
df["interceptions"]=(full.interceptions_x/full.minutes_x*90)/(full.interceptions_y/full.minutes_y*90)*(full.interceptions_y.mean()/full.minutes_y*90)
df["ball_recoveries"]=(full.ball_recoveries_x/full.minutes_x*90)/(full.ball_recoveries_y/full.minutes_y*90)*(full.ball_recoveries_y.mean()/full.minutes_y*90)
temp2=df["pressures"].rank(pct=True)*0.3+df["tackles"].rank(pct=True)*0.25+df["interceptions"].rank(pct=True)*0.25+df["ball_recoveries"].rank(pct=True)*0.2
df["rating_def"]=temp2
df.head()

,player,position,squad,age_full,age,minutes_x,minutes_y,npxg,xa,touches,cross_tend,drib_tend,pr_passes,pr_carries,rating_off,pressures,tackles,interceptions,ball_recoveries,rating_def
0,Tyler Adams,MF,RB Leipzig,23-087,23,1333.0,2970.0,0.007334,0.041335,65.280025,0.004231,0.015238,3.209284,4.697918,0.387460,2.353405,0.236712,1.621982,10.354396,0.488592
1,Amine Adli,"FW,MF",Leverkusen,22-002,22,1256.0,2970.0,0.222477,0.092401,53.366347,0.064838,0.058901,2.782014,5.492130,0.517678,9.875844,0.732735,1.104827,6.963832,0.698867
2,Robert Andrich,MF,Leverkusen,27-232,27,1786.0,2970.0,0.111239,0.035539,59.045495,0.006775,0.010815,4.255267,2.838577,0.459304,2.497821,0.085882,2.102380,11.870622,0.485275
3,Christopher Antwi-Adjei,"FW,MF",Bochum,28-094,28,1130.0,2970.0,0.218685,0.078363,46.393054,0.148148,0.062500,1.849667,5.129054,0.410498,7.354896,0.514382,0.454862,5.531069,0.547006
4,Charles Aránguiz,MF,Leverkusen,33-025,33,1547.0,2970.0,0.062572,0.071078,70.322633,0.010127,0.011290,4.856206,2.041482,0.551173,3.235389,0.694054,2.163358,11.676519,0.760761


In [27]:
df['age'] = df['age'].astype(int)
df_s=df[df["age"]<=20]
df_s.reset_index(drop=True,inplace=True)

In [33]:
#extended scatter function
px = 1/plt.rcParams['figure.dpi']
fig, ax = plt.subplots(figsize=(1600*px, 900*px))

ax.axis('off')

plt.subplots_adjust(left=0.2, bottom=0.2, right=0.8, top=0.8)
grid = plt.GridSpec(3, 3,wspace=0, hspace=0)
ax=fig.add_subplot(grid[:-1, 1:])
ax2=fig.add_subplot(grid[1, 0])
ax3=fig.add_subplot(grid[2, 1])
ax4=fig.add_subplot(grid[0, 0])
ax5=fig.add_subplot(grid[-1, -1])
ax6=fig.add_subplot(grid[2, 0])
ax4.axis('off')
ax5.axis('off')
ax.tick_params(axis="y",direction="in",pad=-20)
ax.tick_params(axis="x",direction="in",pad=-20)
ax2.tick_params(axis="y",direction="in",pad=-20)
ax2.tick_params(axis="x",direction="in",pad=-20)
ax3.tick_params(axis="y",direction="in",pad=-20)
ax3.tick_params(axis="x",direction="in",pad=-20)
ax6.tick_params(axis="y",direction="in",pad=-20)
ax6.tick_params(axis="x",direction="in",pad=-20)
ax2.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax6.spines['top'].set_visible(False)
ax6.spines['right'].set_visible(False)

#first
x=df["rating_off"]
y=df["rating_def"]
ax.scatter(x=x,y=y,c="#39FF14",s=100,alpha=0.3)


for i, txt in enumerate(df_s["player"]):
    ax.annotate(txt, (df_s["rating_off"][i], df_s["rating_def"][i]),size="10")


ax.set_xlim(0.5, 1)
ax.set_ylim(0.5,1)
ax.yaxis.tick_right()
ax.xaxis.tick_top()
ax.xaxis.set_ticks_position('none') 
ax.yaxis.set_ticks_position('none') 
ax.xaxis.set_visible(False) 
ax.yaxis.set_visible(False)

#second
ax2.scatter(x=df["rating_off"],y=df["rating_def"],c="#FADA5E",s=50,alpha=0.3)

for i, txt in enumerate(df_s["player"]):
    ax2.annotate(txt, (df_s["rating_off"][i], df_s["rating_def"][i]),size="6")

ax2.set_xlim(0, 0.5)
ax2.set_ylim(0.5, 1)
ax2.xaxis.set_visible(False) 
ax2.yaxis.set_visible(False)
ax2.axhline(y=0.5, color='white',zorder=1,ls="--",lw=2,alpha=0.6)
ax2.arrow(0.05,0.45, 0,0.45,width=.01*9/16,facecolor="w",alpha=0.6)

#third
ax3.scatter(x=df["rating_off"],y=df["rating_def"],c="#FF69B4",s=50,alpha=0.3)

for i, txt in enumerate(df_s["player"]):
    ax3.annotate(txt, (df_s["rating_off"][i], df_s["rating_def"][i]),size="6")

ax3.set_xlim(0.5, 1)
ax3.set_ylim(0,0.5 )
ax3.xaxis.set_visible(False) 
ax3.yaxis.set_visible(False)
ax3.axvline(x=0.5, color='white',zorder=1,ls="--",lw=2,alpha=0.6)
ax3.arrow(0.45,0.05, 0.45,0,width=.01,facecolor="w",alpha=0.6)

#fourth
ax6.scatter(x=df["rating_off"],y=df["rating_def"],c="grey",s=50,alpha=0.3)
ax6.set_xlim(0,0.5)
ax6.set_ylim(0,0.5 )
ax6.xaxis.set_visible(False) 
ax6.yaxis.set_visible(False)
ax6.arrow(0.05,0.1, 0,0.5,width=0.01*9/16,facecolor="w",alpha=0.6)
ax6.arrow(0.1,0.05, 0.5,0,width=0.01,facecolor="w",alpha=0.6)


transform = ax4.transAxes
rec = plt.Rectangle((0, 0), (1), (1), fill=False, lw=1,transform=transform,color="w",zorder=4)
rec = ax4.add_patch(rec)
rec.set_clip_on(False)
transform = ax5.transAxes
rec = plt.Rectangle((0, 0), (1), (1), fill=False, lw=1,transform=transform,color="w",zorder=4)
rec = ax5.add_patch(rec)
rec.set_clip_on(False)


fig.text(0.5, 0.05, "Info: Min 1000 minutes to be included/players u20 highlighted\nData: FBREF via Statsbomb\nInspired by: @HenshawAnalysis\nMade By: @pawellkaroll", 
                ha = 'center',color = "w", alpha=0.4,
                font={'size':12,"style":"italic"})
fig.text(0.5, 0.95, "BIG 5 Leagues, season 2021/2022", 
                ha = 'center',color = "w", alpha=0.4,
                font={'size':14,'family':"Open Sans","weight":"medium"})
fig.text(0.5, 0.86, "Mezzallas of Europe".upper(), 
                ha = 'center',color = "w", alpha=0.8,
                font={'size':30,'family':"Open Sans","weight":"bold"})
fig_text(x = 0.5, y = 0.83,s="<Searching for ><defensive><, ><offensive>< and ><elite>< midfielders>",ha='center', va='center',
highlight_textprops=[{"color": "white",'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": "#FADA5E",'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": 'white','size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": "#FF69B4",'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": 'white','size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": "#39FF14",'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                {"color": 'white','size':20,'family':"Open Sans","weight":"bold","alpha":0.6}])


fig.text(0.3, 0.225, "OFFENSIVE RATING", 
                ha = 'center',color = "w", alpha=0.6,
                font={'size':14,'family':"Open Sans","weight":"medium"})
fig.text(0.23, 0.25, "DEFENSIVE RATING", 
                ha = 'center',color = "w", alpha=0.6,rotation=90,
                font={'size':14,'family':"Open Sans","weight":"medium"})

txt=ax4.text(0.05, 0.1, "Although being primarly attack-minded player, mezzalla needs to defend high up the pitch. Defensive rating is weighted sum of 4 adjusted statistics: Pressures, Tackles, Interceptions and Ball Recoveries.", 
                ha = 'left',color = "w", alpha=0.6, wrap=True,
                font={'size':12,'family':"Open Sans","weight":"medium"})
txt._get_wrap_line_width = lambda : 300
txt2=ax5.text(0.05, 0.1, "Mezzalla is a player who loves the ball. Generally operates in half-spaces and creates threat from there via pass or carry. As he sometimes drifts wider, mezzala has tendency to cross more often than average central midfielder. Offensive rating is weighted sum of 7 adjusted statistics (see notes).", 
                ha = 'left',color = "w", alpha=0.6, wrap=True,
                font={'size':11,'family':"Open Sans","weight":"medium"})
txt2._get_wrap_line_width = lambda : 300
fig.savefig("mezzallas_scatter.png")